### Transformers

- documentacion y fuentes:
- - https://github.com/codificandobits/Traductor_con_redes_Transformer
- - https://github.com/CyberZHG/keras-transformer
- -  https://arxiv.org/abs/1706.03762

### Diferencias entre secuenciales y transformers

**Secuenciales**:
 - memoria a corto plazo (incluso si son LSTM).
 - Procesado palabra por palabra

 - Memoria a largo plazo usando un mecanismo llamado **atención**
 - procesamiento en paralelo.
  

### Proceso:

<img src = './images/diagrama_transformers.png'>

# Encoding

- Embedding Entrada: Convierte el texto en una serie de vectores, en tokens (representación numerica).


- Codificacion de posicion: La secuencia se procesa en paralelo, es necesario indicar el orden en que se encuentran las palabras en el texto. Este codificador de posiciones genera vectores que se sumaran a los tokens y que indican la posicion relativa de cada token dentro de la secuencia.

(pueden existir varios :))
- **Bloque de codificacion**.
  - - **atencional**: se encarga de analizar la totalidad de la secuencia y buscar relaciones a diferentes niveles dentro de la secuencia.
Los tokens se llevan a 3 redes neuronales entrenadas para encontrar 3 vectores. **Queries**, **keys** y **values**. (3 representaciones alternativas de los tokes originales).

Luego cada queries se compara con los keys, con esto se obtiene un puntaje para el grado de asociacion de pares de palabras 
(por ejemplo con similitud de coseno.). Con estos puntajes se pondera cada uno de los vectores **values**
<center>
<img src = 'images/atencional.png'>
</center>

Es necesario escalar los puntajes, escalandolos dividiendolo entre el tamaño de cada vector y llevandolo a una función softmax.

<center>
<img src = './images/diagrama_transformers_2.png'>
</center>

Luego se multiplica esta matrix con la matriz de values. Estos serán (tantos como palabras hay) nuevos tokes con la codificación de la información de contexto más relevante para cada palabra de la secuencia.

**nota:** Un bloque atencional no es suficiente. Ya que por ejemplo "I Love Euclidean Geometry" puede tener varias asociaciones en un bloque atencional.  [I Love] [Euclidean Geometry] o [I] [Love] [Euclidean Geometry]. Es decir además de asociaciones entre palabras hay asociaciones entre frases.

Si se usan multiples bloques atencionales, es posible encontrar asociaciones de palabras y frases en diferente niveles. Luego si se tienen $k$ bloques atencionales, estos se combinan con una red neuronal en un único vector por cada token

<center>
<img src = './images/diagrama_transformers_3.png'>
</center>



  - -  **bloque residual:** En este bloque se lleva tanto la entrada como la salida del bloque atencional. Luego suma y normaliza estos datos para tener una escala apropiada para el siguiente bloque.

  <center>
<img src = './images/diagrama_transformers_4.png'>
</center>


  - - **red neuronal + bloque residual:**
  La red neuronal procesa en paralelo todos los vectores de la secuencia. tomanda la info atencional de las capas anteriores y consolidandolas en una única representación. La entrada y salida de esta red neuronal son enviadas a un bloque residual que tiene las mismas caracteristicas del bloque anterior

  






# Decoding

<center>
<img src = 'images/decodificador_1.png'>
</center>


- **Embedding Salida** y **Codificación posicional**. (analogo a los anteriores anterior)

- **Bloque Atencional con enmascaramiento** Codifica la relación entre diferentes elementos de salida, usando los **Queries** , **Keys** y **Values**, la gran diferencia es que cada palabra se genera de manera secuencial, el decodificador solo debe prestar atención debe prestar atención **unicamente** alas palabras generas anteriormente y no a las futuras

<center>
<img src = 'images/decodificador_2.png'>
</center>

- **Bloque atencional (decodificador)** A diferencia del bloque atencional anterior (codificación) este bloque centra su atención en la secuencia original como la de salida. Toma la salida del codificador a las redes **Queries** y **Keys**. Mientras que la red **values** tiene como entrada el dato proveniente del bloque residual anterior.

Asi el codificar le comunica al decodificador a que elementos debe prestar mas atencios al momemnto de generar la secuencia de salida (analogamente se toman muchos bloques atencionales para codificar asociaciones). Luego de pasar por los decodificadores
genera un vector con cantidades numericas.

**Capa Lineal** Toma el resultado del decodificador y lo transforma en un vector más grande. Por ejemplo si el traductor aprende 10.000 palabras, entonces el vector de salida de la capa lineal tendra 10.000 elementos

**Softmax** Esta capa convierte cada elemento de este vector en sus respectivas probabilidades.

In [ ]:
import numpy as np 
from keras_transformer import get_model, decode 
from pickle import load  